In [1]:
import sys
sys.path.insert(0,'..')
from paths import *
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import warnings
from tqdm import tqdm

from IPython.display import clear_output


/Users/karthiksoman/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
def remove_outliers(data_, analyte):
    analyte_concentration_values = data_[analyte]
    # Calculate quartiles and IQR
    q1 = analyte_concentration_values.quantile(0.25)
    q3 = analyte_concentration_values.quantile(0.75)
    iqr = q3 - q1
    # Define Tukey's fences
    k = 3
    lower_fence = q1 - k * iqr
    upper_fence = q3 + k * iqr
    # Filter non-outliers
    non_outliers = data_[(data_[analyte] >= lower_fence) & (data_[analyte] <= upper_fence)]
    house_counts = non_outliers.groupby("house").size()
    # Filter to keep only the rows with paired house IDs
    paired_houses = house_counts[house_counts == 2].index
    non_outliers = non_outliers[non_outliers["house"].isin(paired_houses)]
    return non_outliers
    
def glm_model(data_, analyte_, sample_, selected_metadata_columns, remove_outlier):
    data_analyte = data_[selected_metadata_columns+[analyte_]]
    if remove_outlier:
        data_analyte = remove_outliers(data_analyte, analyte_)        
    data_analyte = data_analyte[[analyte_, "pathway_abundance", "Disease_label", "AGE", "BMI", "Gender_label", "site", "house"]]
    data_analyte['Gender_label'] = data_analyte['Gender_label'].astype(int)
    site_dummies = pd.get_dummies(data_analyte['site'], prefix='site', drop_first=True)
    house_dummies = pd.get_dummies(data_analyte['house'], prefix='house', drop_first=True)
    X = sm.add_constant(data_analyte[['AGE', 'BMI', 'Gender_label', 'Disease_label', 'pathway_abundance']])
    X = pd.concat([X, site_dummies, house_dummies], axis=1)
    X = X.astype({col: int for col in X.select_dtypes(include=['bool']).columns})
    X = X.apply(pd.to_numeric)
    try:
        model = sm.GLM(data_analyte[analyte_], X, family=sm.families.Gamma(link=sm.families.links.log()))
        mdf = model.fit()
    except:
        try:
            data_analyte = data_[selected_metadata_columns+[analyte_]]
            data_analyte = remove_outliers(data_analyte, analyte_)
            data_analyte = data_analyte[[analyte_, "pathway_abundance", "Disease_label", "AGE", "BMI", "Gender_label", "site", "house"]]
            data_analyte['Gender_label'] = data_analyte['Gender_label'].astype(int)
            site_dummies = pd.get_dummies(data_analyte['site'], prefix='site', drop_first=True)
            house_dummies = pd.get_dummies(data_analyte['house'], prefix='house', drop_first=True)
            X = sm.add_constant(data_analyte[['AGE', 'BMI', 'Gender_label', 'Disease_label', 'pathway_abundance']])
            X = pd.concat([X, site_dummies, house_dummies], axis=1)
            X = X.astype({col: int for col in X.select_dtypes(include=['bool']).columns})
            X = X.apply(pd.to_numeric)
            model = sm.GLM(data_analyte[analyte_], X, family=sm.families.Gamma(link=sm.families.links.log()))
            mdf = model.fit()
        except:            
            mdf = None
    out_dict = {}
    out_dict["analyte"] = analyte_
    out_dict["sample"] = sample_
    out_dict["patient_sample_count"] = data_analyte.shape[0]
    out_dict["model"] = mdf
    return out_dict 
    
def run_model(pathway, category, sample):
    if category != 'full':
        ms_category_data_ = ms_category_data[ms_category_data['disease_course_control'] == category]
    else:
        ms_category_data_ = ms_category_data.copy()
    pathway_data_sel = pathway_data[pathway_data['# Pathway'] == pathway]
    values = pathway_data_sel.iloc[0, 1:].values
    columns = pathway_data_sel.columns[1:]
    pathway_data_sel = pd.DataFrame({'pathway_abundance': values, 'CLIENT_SAMPLE_ID': columns})

    if sample == "serum":
        filename = GLOBAL_SERUM_DATA_FILENAME
    else:
        filename = GLOBAL_STOOL_DATA_FILENAME
        
    file_path = os.path.join(DATA_ROOT_PATH, filename)
    sheet_name = ["Chemical Annotation", "Sample Meta Data", "Log Transformed Data"]
    analyte_metadata = pd.read_excel(file_path, engine='openpyxl', sheet_name=sheet_name[0])
    patient_metadata = pd.read_excel(file_path, engine='openpyxl', sheet_name=sheet_name[1])
    data = pd.read_excel(file_path, engine='openpyxl', sheet_name=sheet_name[2])
    patient_metadata.loc[:, "site_code"] = patient_metadata["CLIENT_SAMPLE_ID"].apply(lambda x:x[0:3])
    global_metabolomics_compound_spoke_map = pd.read_csv(os.path.join(os.path.dirname(OUTPUT_PATH), "global_metabolomics_compound_spoke_map.csv"))

    analyte_columns = list(data.columns)
    analyte_columns.remove("PARENT_SAMPLE_NAME")
    
    analyte_columns_selected = global_metabolomics_compound_spoke_map[global_metabolomics_compound_spoke_map.CHEM_ID.isin(analyte_columns)]["CHEM_ID"].unique()
    
    data_with_analyte_columns_selected = data[["PARENT_SAMPLE_NAME"]+list(analyte_columns_selected)]
    selected_metadata_columns = ["PARENT_SAMPLE_NAME", "CLIENT_IDENTIFIER", "GROUP_NAME", "AGE", "BMI", "GENDER", "CLIENT_SAMPLE_ID", "CLIENT_MATRIX", "TREATMENT", "SAMPLE_AMOUNT_UNITS"]
    patient_metadata_selected_columns = patient_metadata[selected_metadata_columns]
    patient_metadata_selected_columns.loc[:, 'house'] = (patient_metadata_selected_columns['CLIENT_SAMPLE_ID'].str[:3] + patient_metadata_selected_columns['CLIENT_SAMPLE_ID'].str[-4:])
    patient_metadata_selected_columns.loc[:, 'site'] = patient_metadata_selected_columns.loc[:, 'CLIENT_SAMPLE_ID'].str[:3]
    
    
    data_with_patient_metadata = pd.merge(data_with_analyte_columns_selected, patient_metadata_selected_columns, on="PARENT_SAMPLE_NAME")
    
    selected_metadata_columns.append('site')
    selected_metadata_columns.append('house')
    
    clear_output()
    ila_chem_id = analyte_metadata[analyte_metadata['CHEMICAL_NAME'].str.startswith('indolelactate')].CHEM_ID.values[0]
    iaa_chem_id = analyte_metadata[analyte_metadata['CHEMICAL_NAME'].str.startswith('indoleacetate')].CHEM_ID.values[0]
    ila_iaa_imsms_data = data_with_patient_metadata[['CLIENT_SAMPLE_ID', ila_chem_id, iaa_chem_id, 'GENDER', 'BMI', 'AGE', 'site', 'house', 'PARENT_SAMPLE_NAME', 'CLIENT_IDENTIFIER', 'GROUP_NAME', 'CLIENT_MATRIX', 'TREATMENT', 'SAMPLE_AMOUNT_UNITS']]
    pathway_and_imsms_data = pd.merge(ila_iaa_imsms_data, pathway_data_sel, on='CLIENT_SAMPLE_ID')
    pathway_and_imsms_data = pathway_and_imsms_data.rename(columns={ila_chem_id:'exp_ILA', iaa_chem_id:'exp_IAA'})
    pathway_and_imsms_data.loc[:,'exp_ratio'] = pathway_and_imsms_data.exp_ILA - pathway_and_imsms_data.exp_IAA
    pathway_and_imsms_data_reverse_log_transformed = pathway_and_imsms_data.copy()
    analyte_columns_selected = ['exp_ILA', 'exp_IAA', 'exp_ratio']
    for analyte in analyte_columns_selected:
        analyte_concentration = pathway_and_imsms_data_reverse_log_transformed[analyte].values
        pathway_and_imsms_data_reverse_log_transformed.loc[:, analyte] = np.exp(analyte_concentration)
    
    clear_output()
    pathway_and_imsms_data_reverse_log_transformed.loc[:, 'house'] = (pathway_and_imsms_data_reverse_log_transformed['CLIENT_SAMPLE_ID'].str[:3] + pathway_and_imsms_data_reverse_log_transformed['CLIENT_SAMPLE_ID'].str[-4:])
    pathway_and_imsms_data_reverse_log_transformed.loc[:, 'site'] = pathway_and_imsms_data_reverse_log_transformed.loc[:, 'CLIENT_SAMPLE_ID'].str[:3]
    
    le = LabelEncoder()
    
    pathway_and_imsms_data_reverse_log_transformed.loc[:, 'Gender_label'] = le.fit_transform(pathway_and_imsms_data_reverse_log_transformed['GENDER'])
    pathway_and_imsms_data_reverse_log_transformed.loc[:, 'Disease_label'] = le.fit_transform(pathway_and_imsms_data_reverse_log_transformed['GROUP_NAME'])
    
    house_to_exclude = pathway_and_imsms_data_reverse_log_transformed[pathway_and_imsms_data_reverse_log_transformed.isna().any(axis=1)].house.values
    
    pathway_and_imsms_data_reverse_log_transformed = pathway_and_imsms_data_reverse_log_transformed[~pathway_and_imsms_data_reverse_log_transformed["house"].isin(house_to_exclude)]
    
    selected_metadata_columns.extend(['pathway_abundance', 'Gender_label', 'Disease_label'])
    
    pathway_and_imsms_data_reverse_log_transformed = pathway_and_imsms_data_reverse_log_transformed[pathway_and_imsms_data_reverse_log_transformed.house.isin(ms_category_data_.household.astype(str))]
    model_dict_list = []
    for analyte_column_selected in tqdm(analyte_columns_selected):
        model_dict_list.append(glm_model(pathway_and_imsms_data_reverse_log_transformed,
                  analyte_column_selected,
                  sample,
                  selected_metadata_columns,
                  remove_outlier=False
                 ))
    clear_output()
    if sample == "serum":
        filename = GLOBAL_SERUM_DATA_FILENAME
    else:
        filename = GLOBAL_STOOL_DATA_FILENAME
    file_path = os.path.join(DATA_ROOT_PATH, filename)
    sheet_name = ["Chemical Annotation", "Sample Meta Data", "Log Transformed Data"]
    analyte_metadata = pd.read_excel(file_path, engine='openpyxl', sheet_name=sheet_name[0])
    
    
    model_summary_list_of_df = []
    for index, model in enumerate(model_dict_list):
        model_summary_list = []
        analyte_name = model["analyte"]
        try:            
            disease_coeff = model["model"].params['pathway_abundance']
            disease_coeff_pvalue = model["model"].pvalues['pathway_abundance']
            disease_coeff_CI = tuple(model["model"].conf_int().loc['pathway_abundance'])
            model_converged_flag = model["model"].converged
            N = model['patient_sample_count']
        except:
            disease_coeff = None
            disease_coeff_pvalue = None
            disease_coeff_CI = None
            model_converged_flag = None
            N = model['patient_sample_count']
        model_summary_list.append((analyte_name, disease_coeff, disease_coeff_pvalue, disease_coeff_CI, model_converged_flag, N))        
        model_summary_list_of_df.append(pd.DataFrame(model_summary_list, columns=["analyte_name", "pathway_abundance_coeff", "pvalue", "CI", "model_converged_flag", "number_of_samples"]))
    model_summary = pd.concat(model_summary_list_of_df, ignore_index=True)
    model_summary.loc[:, 'analyte_type'] = 'untargeted'
    return model_summary



In [3]:
category_list  = ['full', 'RRMS', 'PMS']
sample_list = ['feces', 'serum']
pathway_list = ['P161-PWY: acetylene degradation']


In [4]:
pathway_data = pd.read_excel(os.path.join(DATA_ROOT_PATH, 'john_hopkins/1-s2.0-S0092867422011151-mmc6.xlsx'), engine='openpyxl', sheet_name='pathway')
ms_category_data = pd.read_excel(os.path.join(DATA_ROOT_PATH, 'john_hopkins/1-s2.0-S0092867422011151-mmc2.xlsx'), engine='openpyxl', sheet_name='Sample phenotype')


In [5]:
%%time

model_out = []
for pathway in tqdm(pathway_list):
    for category in category_list:
        for sample in sample_list:
            model_summary = run_model(pathway, category, sample)
            out_dict = {
                "pathway" : pathway,
                "cohort_category" : category,
                "sample" : sample,
                "model_summary" : model_summary
            }
            model_out.append(out_dict)


100%|█████████████████████████████████████████████| 1/1 [01:39<00:00, 99.89s/it]

CPU times: user 2min 9s, sys: 1min 24s, total: 3min 33s
Wall time: 1min 39s


In [6]:
import joblib

with open(os.path.join(DATA_ROOT_PATH, f'john_hopkins/glm_model_results_for_pathway.joblib'), 'wb') as f:
    joblib.dump(model_out, f)
    